In [112]:
import numpy as np
import pandas as pd
import json
import voila
import http.client, urllib.parse

In [57]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-11-05


In [113]:
!jupyter serverextension enable voila --sys-prefix

Enabling: voila
- Writing config: C:\Users\user\anaconda3\etc\jupyter
    - Validating...
      voila 0.4.0 ok


In [59]:
# for x in dict_data['data']:
#     df = pd.DataFrame.from_dict(x, orient='index',
#                        columns=['author', 'title', 'description', 'url','source','image','category','language','country','published_at'])

In [60]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [61]:
# Image Widget

file = open("download.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='300'
                )

label_headline = widgets.Label(
                    value='Image by Iconka',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [62]:
vbox_headline

In [63]:
# news source

source = widgets.Dropdown(
    options=[('General',''),('BBC', 'bbc'), ('CNN', 'cnn'), ('ESPN', 'espn')],
    value='',
    description='Source:',
)

In [64]:
# category

category = widgets.Dropdown(
    options=[('General', 'general'), ('Business', 'business'),('Entertainment','entertainment'),('Health', 'health'),('Science','science'),('Technology','technology')],
    value='general',
    description='Category:',
)

In [65]:
fill = widgets.Text(
    value='',
    placeholder='Enter Keyword(s)',
    description='Keyword:',
    disabled=False
)

In [66]:
#Date picker

date_start = widgets.DatePicker(value = today, description='Pick a Start')
date_end = widgets.DatePicker(value = today,description='Pick an End')

In [67]:
button_send = widgets.Button(
                description='Confirm',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print("Loading...")
        print("Fetching" ,category.value, "news from", source.value, "with",fill.value, "as a keyword from", date_start.value,"to",date_end.value)

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [68]:
vbox_text = widgets.VBox([source, category, fill, date_start, date_end, vbox_result])

In [69]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)

In [101]:
temp_start = date_start.value.strftime('%Y-%m-%d')
temp_end = date_end.value.strftime('%Y-%m-%d')
parsed_date = temp_start +','+temp_end

In [102]:
parsed_date

'2022-11-01,2022-11-02'

In [105]:
ACCESS_KEY = '243247ebaf607124b6b062d06e319452'

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': ACCESS_KEY,
    'languages' : 'en',
    'limit' : '100',
    'date' : parsed_date,
    'categories' : category.value,
    'keywords' : fill.value,
    'sources' : source.value,
    'sort' : 'published_desc'
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

print(data.decode('utf-8'))

{"pagination":{"limit":100,"offset":0,"count":100,"total":10000},"data":[{"author":"The Canadian Press","title":"Three Chinese companies ordered to divest as Ottawa moves to protect critical mineral industry","description":"Canada and its allies are desperately trying to upend China\u0027s dominance in the field and create a more stable and reliable supply chain","url":"https:\/\/nationalpost.com\/news\/politics\/federal-government-moves-to-cut-china-out-of-canadian-critical-mineral-industry","source":"nationalpost","image":null,"category":"general","language":"en","country":"us","published_at":"2022-11-02T23:59:17+00:00"},{"author":"newsfeeds@nzherald.co.nz","title":"M\u0026#257;ori communities criticise lack of consultation in report on areas at risk of flooding","description":"M\u0026#257;ori communities are worried the government\u0027s climate planning could override their own work and rangatiratanga.A government report has for the first time identified 44 unprepared communities a

In [106]:
dict_data = json.loads(data.decode('utf-8'))

In [107]:
if(dict_data['pagination']['count'] != 0):
    df = pd.DataFrame()
    for x in dict_data['data']:
        df = df.append(x, ignore_index=True)
    
elif (dict_data['pagination']['count'] == 0):
    print("Cancel")

In [108]:
title_df = df['title']

In [109]:
title_df

0     Three Chinese companies ordered to divest as O...
1     M&#257;ori communities criticise lack of consu...
2     Michigan man gets 70 to 100 years for hammer m...
3     San Diego sax great Anthony Ortega, dead at 94...
4     Estas son todas las propuestas iberoamericanas...
                            ...                        
95    Chris Perkins: Dolphins’ Tua talks about Super...
96    Montana National Guard volunteers receive nati...
97    Chris Perkins: Dolphins’ Tua talks about Super...
98    Snack Drawer Bolsters Team With Three New Hire...
99    AC Milan, Leipzig take final last-16 places in...
Name: title, Length: 100, dtype: object